# 镶嵌栅格

In [2]:
import os
import arcpy
from arcpy import env
from arcpy.sa import *
import re

In [3]:
raster_path = r"F:\ArcgisData\m5\ky"

In [4]:
raster_list = []
for roots,dirs,files in os.walk(raster_path):
    for one_file in files:
        if one_file.endswith(".bil") or one_file.endswith(".BIL"):
            raster_list.append(os.path.join(roots,one_file))
# 使用正则表达式匹配下划线和点号之间的数字
def find_number(path):
    match = re.findall(r'_(\d+)\.', path)
    return str(match[0]).zfill(3)
# raster_list = sorted(raster_list,key=find_number)

In [5]:
raster_list,len(raster_list)

(['F:\\ArcgisData\\m5\\ky\\G48G015075\\G48G015075.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015076\\G48G015076.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015077\\G48G015077.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015078\\G48G015078.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015079\\G48G015079.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015080\\G48G015080.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015081\\G48G015081.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015082\\G48G015082.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015083\\G48G015083.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015084\\G48G015084.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015085\\G48G015085.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015086\\G48G015086.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G015087\\G48G015087.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G016075\\G48G016075.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G016076\\G48G016076.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G016077\\G48G016077.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G016078\\G48G016078.bil',
  'F:\\ArcgisData\\m5\\ky\\G48G016079\\G48G01607

In [6]:
# 镶嵌至新栅格
def mosaic_new_raster(raster_list,out_gdb,out_name,pre_refer,bands:int):
    pre_refer_prj = arcpy.Describe(pre_refer).spatialReference
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        arcpy.management.MosaicToNewRaster(raster_list, out_gdb, out_name, pre_refer_prj, "32_BIT_FLOAT", None, bands, "LAST", "FIRST")

In [7]:
out_gdb = r"F:\ArcgisData\tif_file"
out_name = "ky_dem.tif"
pre_fer = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"

In [8]:

mosaic_new_raster(raster_list,out_gdb,out_name,pre_fer,1)

In [19]:
# 处理异常值
def Con_calc(raster,error_list):
    # 按焦点统计


    # 识别异常值
    outCon1 = Con((Raster(raster) >error_list[0]) | (Raster(raster) <=error_list[1]), 1, 0)

    # 使用焦点统计来处理异常值
    neighborhood = NbrCircle(3, "CELL")  # 设置邻域大小和形状
    # outFocalStat = FocalStatistics(raster, neighborhood, "MINIMUM")
    outFocalStat = FocalStatistics(raster, neighborhood, "MEAN")

    # 替换异常值
    outRaster = Con(outCon1 == 1, outFocalStat, raster)
    return outRaster
# 取消并行处理
def disable_parallel_processing(func):
    """"装饰器函数,取消并行处理,因为在部分PC中无法进行并行处理"""
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper

# 填洼
@disable_parallel_processing
def fill_dem(dem_raster):
    """参数默认为最佳"""
    fill_raster = Fill(dem_raster,)
    return fill_raster
    

In [12]:
env.workspace = out_gdb

In [16]:
dem_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM2"
temp = Con_calc(dem_path,[4000,0])
temp.save('DEM3')

In [20]:
# 填洼
fill_dem('DEM3').save('DEM4')